In [13]:
import requests

from bs4 import BeautifulSoup
import pandas as pd
from tqdm.auto import tqdm

In [2]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en;q=0.5",
    "Referer": "https://www.google.com/",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0"
}

In [3]:
carrefour_url = "https://www.carrefour.fr/r/boissons/cave-a-vins"

In [4]:
request = requests.get(carrefour_url, headers=headers)

In [5]:
soup = BeautifulSoup(request.text)

In [6]:
products = soup.find_all("article")

In [7]:
carrefour_wines = dict()
for index, product in enumerate(products):
  try:
    name = product.find("h2").text.strip()
    price_per_liter = product.find("div", {"class": "ds-body-text ds-body-text--size-s ds-body-text--color-standard-3 product-card-per-unit-label"})
    price_per_liter = float(price_per_liter.text.split(" €", 1)[0].strip())
    price = product.find("div", {"class": "product-card-price__price"})
    price = float(price.text.split("€", 1)[0].strip().replace(",", "."))
    carrefour_wines[name] = {"price": price, "price / l": price_per_liter}
  except AttributeError:
    pass

**Vivino**

In [8]:
def clean_wine_name(stringa: str):
  """
  Return str without useless text to enhance vivino search.
  """
  clean_wine_name = stringa.lower()
  clean_wine_name = stringa.replace("vin blanc ", "")
  clean_wine_name = clean_wine_name.replace("vin rouge ", "")
  clean_wine_name = clean_wine_name.replace("vin rosé ", "")
  return clean_wine_name

In [18]:
def scrape_vivino(wine_name):
  base_url = "https://www.vivino.com/search/wines?q="
  url = base_url + clean_wine_name(wine_name.replace(" ", "+"))
  response = requests.get(url,
                            headers=headers)
  soup = BeautifulSoup(response.text)
  wine = soup.find("div", {"class": "default-wine-card"})
  
  name = wine.find("span", {"class": "header-smaller text-block wine-card__name"}).text.strip()
  price = wine.find("span", {"class": "wine-price-value"}).text.strip().replace("—", "")
  rating = float(wine.find("div", {"class": "text-inline-block light average__number"}).text.strip())
  n_reviews = int(wine.find("p", {"class": 'text-micro'}).text.split(" ", 1)[0].strip())

  return {"VivinoName": name,
          "VivinoPrice": price,
          "Rating": rating,
          "# Reviews": n_reviews}

In [19]:
result = dict()
wines = list(carrefour_wines.keys())[:4]
for wine_name in tqdm(wines):
  temp = carrefour_wines[wine_name]
  temp.update(scrape_vivino(wine_name))
  result[wine_name] = temp

  0%|          | 0/4 [00:00<?, ?it/s]

In [25]:
df = pd.DataFrame.from_dict(result, orient='index')

In [27]:
df.to_excel("carrefour_wines.xlsx")

try:
  # if we are on Colab
  from google.colab import files
  files.download("carrefour_wines.xlsx")
except ModuleNotFoundError:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>